# Imports

In [ ]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path
import json

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories
)

from model_ddp.simulations.gaussian_simulator import (
    RegressionSimulator,
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    TransformationFunctions
)

# Modelos
from model_ddp.models.LSBP_normal_v1 import LSBPNormal

# Metricas
from model_ddp.fit.metrics import regression_metrics

config=load_config()

# Flujo 

- Datos: Se guardaran todo en una carpeta. Por tanto todo tratamiento debe ir previo a un experimento.
- Experimentos: Cada modelo quedara en una carpeta con id, por tanto se crean versiones para cada uno.

In [ ]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_lsbp_001"
SIM_REAL = "simulation"

In [ ]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{NOMBRE_EJECUCION}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=50,
    n_features=5,
    x_range=(0.0, 10.0),
    noise_std=0.3,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=2.0,
    variance=1.0
)

# Definir transformación (regresión lineal)
coefficients = np.array([2.5, -1.8, 0.9, 1.2, -0.5])
intercept = 3.0

transformation = TransformationFunctions.linear(
    coefficients=coefficients,
    intercept=intercept
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

## Experimento I 

In [ ]:
# Parámetros de ejecución
CARACTERISTICAS = "Test inicial: GP con kernel RBF + transformación lineal"
EXPERIMENT_ID = create_experiment_id("lsbp_001")

In [ ]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
lsbp_model = LSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = lsbp_model.run(
    iterations=50,          # Iteraciones totales
    burnin=10               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)

In [ ]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto LSBPNormal)
model_file = carpeta_modelo / "lsbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(lsbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = lsbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir tuplas a listas para JSON
    summary_json = {k: {'mean': v[0], 'std': v[1]} for k, v in summary.items()}
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'LSBPNormal',
    'data_shape': {
        'n': lsbp_model.n,
        'p': lsbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': lsbp_model.n_grid
    },
    'priors': {
        'mu_prior': (lsbp_model.mu_mu, lsbp_model.tau_mu_inv),
        'mu0_prior': (lsbp_model.m0, lsbp_model.s02),
        'kappa0_prior': (lsbp_model.alpha_kappa, lsbp_model.beta_kappa),
        'a0_prior': (lsbp_model.alpha_a, lsbp_model.beta_a),
        'b0_prior': (lsbp_model.alpha_b, lsbp_model.beta_b),
        'psi_prior': (lsbp_model.mu_psi, lsbp_model.tau_psi_inv)
    },
    'final_stats': {
        'H_final': lsbp_model.H,
        'n_clusters_mean': summary['n_clusters'][0],
        'n_clusters_std': summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': np.mean(lsbp_model.mh_acceptance['alpha'][-100:]) if lsbp_model.mh_acceptance['alpha'] else 0,
        'psi': np.mean(lsbp_model.mh_acceptance['psi'][-100:]) if lsbp_model.mh_acceptance['psi'] else 0,
        'kappa0': np.mean(lsbp_model.mh_acceptance['kappa0'][-100:]) if lsbp_model.mh_acceptance['kappa0'] else 0,
        'a0': np.mean(lsbp_model.mh_acceptance['a0'][-100:]) if lsbp_model.mh_acceptance['a0'] else 0
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': lsbp_model.y_mean,
    'y_std': lsbp_model.y_std,
    'X_mean': lsbp_model.X_mean,
    'X_std': lsbp_model.X_std
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)

In [ ]:
##################################################
# Graficas y FIT  (SEGUN EXPERIMENTO)
##################################################
# FALTA ESTO

metrics = regression_metrics(y_true, y_pred)

print(metrics)



In [ ]:
##################################################
# Reguistrar exprimento 
##################################################

# Y ESTO 